## качаем данные

In [1]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm

In [2]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]
    
urls = []
for page in tqdm(range(1, 5000), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))

Pages:   0%|          | 0/4999 [00:00<?, ?it/s]

ConnectionError: HTTPSConnectionPool(host='zpp.rospotrebnadzor.ru', port=443): Max retries exceeded with url: /Forum/Appeals/AjaxindexList?page=900&searchtext=&categories=%5B%5D (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x107a1eeb0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [3]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')

Pages:   0%|          | 0/4495 [00:00<?, ?it/s]

ConnectionError: HTTPSConnectionPool(host='zpp.rospotrebnadzor.ru', port=443): Max retries exceeded with url: /Forum/Appeals/Details/33341 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x110eda070>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [7]:
print('Q: ',len(question),'A: ', len(question))

Q:  344 A:  344


In [4]:
## посмотрим, что скачалось
N = 2
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Добрый день,

19 июня 2022 года мною был куплен товар Мясорубка в павильоне #335 рынка Юнона.  При этом продавцы утверждали, что товар отличного качества. При использовании товара выяснилось, что нож в составе товара очень плохой, вследствие чего мясо не прокручивается, к тому же мясорубка окрашивает мясо (у меня маленькие дети). 

1 июля была предпринята попытка возврата товара как некачественного, в чем было отказано, т. к. чек был мной утрачен. Имеется свидетельство оплаты от банка (прикреплено). 

Прошу разобраться с продавцом и предоставить мне возможность вернуть товар. В соответствии со ст. 493 Гражданского кодекса я имею право сделать это без бумажного чека. 



A: Уважаемая Анна!
Разъясняю, что в соответствии с п. 1, п. 5 статьи 18 Закона РФ от 07.02.1992 г. № 2300-1 «О защите прав потребителей» (далее – «Закон РФ»), потребитель в случае обнаружения в товаре недостатков, если они не были оговорены продавцом, по своему выбору вправе: потребовать замены на товар этой же марки

In [10]:
import json
with open('question.json', 'w') as f:
    json.dump(question, f)

In [11]:
with open('answers.json', 'w') as f:
    json.dump(answers, f)